In [1]:
pip install mido python-rtmidi


Note: you may need to restart the kernel to use updated packages.


In [2]:
from mido import Message, MidiFile, MidiTrack
import math

# Jacob's vibrational frequency mapping
vibration_freq = {
    'A': 329.628, 'R': 138.591, 'N': 82.4069, 'D': 246.924, 'C': 293.665,
    'Q': 146.832, 'E': 110, 'G': 369.994, 'H': 146.832, 'I': 293.665,
    'L': 77.7817, 'K': 246.942, 'M': 77.7817, 'F': 164.814, 'P': 116.541,
    'S': 195.998, 'T': 293.665, 'W': 130.813, 'Y': 61.7354, 'V': 246.942
}

# Convert freq (Hz) to MIDI note
def freq_to_midi_note(freq):
    return int(round(69 + 12 * math.log2(freq / 440.0)))

# Create track
def create_track(sequence, instrument, name=""):
    track = MidiTrack()
    track.append(Message('program_change', program=instrument, time=0))
    for aa in sequence:
        freq = vibration_freq.get(aa)
        if freq:
            note = freq_to_midi_note(freq)
            track.append(Message('note_on', note=note, velocity=64, time=0))
            track.append(Message('note_off', note=note, velocity=64, time=480))
    return track

# Nanobody 2VYR
seq_2vyr = "EVQLLESGGGLVQPGGSLRLSCAASGFTFEEYAMLWVRQAPGKGLEWVSGINARGYTTYYADSVKGRFTISRDNSKNTLYLQMNSLRTEDTAVYYCAKPWYPFMASKGSEFDYWGQGTLVTVSSAAALEIKRASQPELAPEDPEDVEHHHHHH"
fr2_2vyr = "WVRQAPGKGLEWVSGINARGY"  # placeholder from ModiBodies
cdr3_2vyr = "CAKPWYPFMASKGSEFDY"     # placeholder from ModiBodies

# Create MIDI
mid = MidiFile()
mid.tracks.append(create_track(seq_2vyr, instrument=0))       # Piano 🎹
mid.tracks.append(create_track(fr2_2vyr, instrument=24))      # Guitar 🎸
mid.tracks.append(create_track(cdr3_2vyr, instrument=114))    # Drums 🥁

# Save
mid.save("2VYR_music.mid")


In [2]:
from mido import Message, MidiFile, MidiTrack
import math

# Jacob's vibrational frequency mapping
vibration_freq = {
    'A': 329.628, 'R': 138.591, 'N': 82.4069, 'D': 246.924, 'C': 293.665,
    'Q': 146.832, 'E': 110, 'G': 369.994, 'H': 146.832, 'I': 293.665,
    'L': 77.7817, 'K': 246.942, 'M': 77.7817, 'F': 164.814, 'P': 116.541,
    'S': 195.998, 'T': 293.665, 'W': 130.813, 'Y': 61.7354, 'V': 246.942
}

# Convert freq (Hz) to MIDI note
def freq_to_midi_note(freq):
    return int(round(69 + 12 * math.log2(freq / 440.0)))

# Create track
def create_track(sequence, instrument, name=""):
    track = MidiTrack()
    track.append(Message('program_change', program=instrument, time=0))
    for aa in sequence:
        freq = vibration_freq.get(aa)
        if freq:
            note = freq_to_midi_note(freq)
            track.append(Message('note_on', note=note, velocity=64, time=0))
            track.append(Message('note_off', note=note, velocity=64, time=480))
    return track

# Nanobody 2VYR
seq_2vyr = "EVQLLESGGGLVQPGGSLRLSCAASGFTFEEYAMLWVRQAPGKGLEWVSGINARGYTTYYADSVKGRFTISRDNSKNTLYLQMNSLRTEDTAVYYCAKPWYPFMASKGSEFDYWGQGTLVTVSSAAALEIKRASQPELAPEDPEDVEHHHHHH"
fr2_2vyr = "WVRQAPGKGLEWVSGINARGY"  # placeholder from ModiBodies
cdr3_2vyr = "CAKPWYPFMASKGSEFDY"     # placeholder from ModiBodies

# Create MIDI
mid = MidiFile()
mid.tracks.append(create_track(seq_2vyr, instrument=0))       # Piano 🎹
mid.tracks.append(create_track(fr2_2vyr, instrument=24))      # Guitar 🎸
mid.tracks.append(create_track(cdr3_2vyr, instrument=114))    # Drums 🥁

# Save
mid.save("2VYR_music.mid")
